<a href="https://colab.research.google.com/github/samunderSingh12/debate_baby/blob/main/debate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Install Libraries
!pip install -q gradio openai


In [2]:
# @title 2. Import Libraries and Define Functions (with Enhanced Interaction Prompts)

import gradio as gr
import openai
import time
import textwrap


DEFAULT_OPENROUTER_MODEL = "mistralai/mistral-7b-instruct:free"
DEFAULT_OPENAI_MODEL = "gpt-3.5-turbo"
DEFAULT_MAX_TOKENS = 300 # Slightly increased default
DEFAULT_DEBATE_TURNS = 3
TEMPERATURE = 0.7


DEFAULT_OPENAI_SYS_PROMPT = (
    "You are a precise and analytical AI debater representing the OpenAI perspective. "
    "Engage directly with your opponent's arguments, referencing specific points they've made throughout the debate where relevant. "
    "Maintain a logical flow and build upon your previous arguments. Your goal is a constructive exchange of ideas."
)
DEFAULT_OPENROUTER_SYS_PROMPT = (
    "You are a creative and insightful AI debater representing the OpenRouter perspective. "
    "Challenge your opponent's points thoughtfully and connect your arguments back to the core topic. "
    "Feel free to refer to earlier statements in the debate to highlight consistencies or contradictions. Aim for a compelling and engaging discussion."
)


def get_openai_client(api_key):
    if not api_key: raise gr.Error("OpenAI API Key is missing!")
    try:
        client = openai.OpenAI(api_key=api_key)
        client.models.list(); return client
    except openai.AuthenticationError: raise gr.Error("Invalid OpenAI API Key.")
    except Exception as e: raise gr.Error(f"OpenAI Client Error: {e}")

def get_openrouter_client(api_key):
    if not api_key: raise gr.Error("OpenRouter API Key is missing!")
    try:
        client = openai.OpenAI(
            base_url="https://openrouter.ai/api/v1", api_key=api_key,
            default_headers={"HTTP-Referer": "http://localhost:7860", "X-Title": "Colab AI Debate v5"},
        )
        client.models.list(); return client
    except openai.AuthenticationError: raise gr.Error("Invalid OpenRouter API Key.")
    except Exception as e: raise gr.Error(f"OpenRouter Client Error: {e}")


def run_debate(topic,
               openai_key, openrouter_key,
               openai_model_name, openrouter_model_name,
               openai_system_prompt, openrouter_system_prompt, # Custom prompts
               max_tokens_input):

    # --- Input Validation (Unchanged) ---
    if not topic: raise gr.Error("Please provide a debate topic!")
    # ... (keep other validations for keys, models, prompts, tokens) ...
    if not openai_key: raise gr.Error("Please provide your OpenAI API Key!")
    if not openrouter_key: raise gr.Error("Please provide your OpenRouter API Key!")
    if not openai_model_name: raise gr.Error("Please provide an OpenAI Model Name!")
    if not openrouter_model_name: raise gr.Error("Please provide an OpenRouter Model Name!")
    if not openai_system_prompt: raise gr.Error("Please provide an OpenAI System Prompt!")
    if not openrouter_system_prompt: raise gr.Error("Please provide an OpenRouter System Prompt!")
    try:
        max_tokens_per_turn = int(max_tokens_input)
        if max_tokens_per_turn <= 0: raise ValueError("Max tokens must be positive.")
    except (ValueError, TypeError):
        raise gr.Error("Invalid Max Tokens value. Please use the slider or enter a positive number.")


    # --- Initialize Clients (Unchanged) ---
    try:
        yield "Initializing Clients...", ""
        openai_client = get_openai_client(openai_key)
        openrouter_client = get_openrouter_client(openrouter_key)
    except gr.Error as e:
        yield f"Initialization Error: {e}", "Error"; return

    # --- Setup Debate (Unchanged logic, using inputs) ---
    model_a_name = f"OpenAI ({openai_model_name})"
    model_b_name = f"OpenRouter ({openrouter_model_name})"
    model_a_client = openai_client
    model_b_client = openrouter_client
    model_a_model_name = openai_model_name
    model_b_model_name = openrouter_model_name
    model_a_system_prompt = openai_system_prompt
    model_b_system_prompt = openrouter_system_prompt

    conversation_history = [] # Stores {'role': 'user'|'assistant', 'content': ...} pairs

    debate_transcript = ( # Header remains same as v4
        f"## Debate Topic: {topic}\n\n"
        f"**Settings:**\n"
        f"- OpenAI Model: `{openai_model_name}`\n"
        f"- OpenRouter Model: `{openrouter_model_name}`\n"
        f"- Max Tokens: {max_tokens_per_turn}\n"
        f"- OpenAI Persona: *{textwrap.shorten(model_a_system_prompt, 100)}*\n"
        f"- OpenRouter Persona: *{textwrap.shorten(model_b_system_prompt, 100)}*\n\n"
        f"---\n\n"
    )
    spinner_updates = ["Thinking.", "Thinking..", "Thinking..."]

    # --- Run Debate Turns ---
    try:
        current_user_instruction_text = ""

        for turn in range(DEFAULT_DEBATE_TURNS * 2):
            is_model_a_turn = turn % 2 == 0
            current_model_name = model_a_name if is_model_a_turn else model_b_name
            current_client = model_a_client if is_model_a_turn else model_b_client
            current_model = model_a_model_name if is_model_a_turn else model_b_model_name
            current_system_prompt = model_a_system_prompt if is_model_a_turn else model_b_system_prompt

            status_message = f"Turn {turn // 2 + 1} / {DEFAULT_DEBATE_TURNS} - {current_model_name} {spinner_updates[turn % 3]}"
            print(f"--- {status_message} ---")
            yield debate_transcript, status_message

            # --- NEW: Refined User Instruction Prompting ---
            if turn == 0:
                # Initial turn instruction remains simple
                current_user_instruction_text = f"Begin the debate by presenting your opening statement on the topic: '{topic}'."
            else:
                # Subsequent turns: Encourage considering history while responding to the last point
                last_message_content = conversation_history[-1]['content'] # Get previous assistant response
                # Subtle change: Frame the request to consider the history implicitly
                current_user_instruction_text = (
                    f"Considering the debate history so far, present your response to the opponent's previous statement. "
                    f"Opponent's statement: '{textwrap.shorten(last_message_content, width=150, placeholder='...')}'"
                )
            # --- End Refined Prompting ---

            # Build messages list dynamically (System + History + Current Instruction)
            messages_for_api_call = [{"role": "system", "content": current_system_prompt}]
            messages_for_api_call.extend(conversation_history) # Add history turns
            messages_for_api_call.append({"role": "user", "content": current_user_instruction_text}) # Add this turn's instruction

            # --- API Call (Memory concern: Check potential token count before sending if needed) ---
            # Note: Simple check, not precise token counting. Models usually handle ~4k-8k+, up to 128k+ tokens.
            approx_msg_count = len(messages_for_api_call)
            if approx_msg_count > 50: # Arbitrary threshold, adjust if needed
                 print(f"Warning: Sending {approx_msg_count} messages, potential context length issue.")

            try:
                print(f"Sending {len(messages_for_api_call)} messages to {current_model_name}")
                response = current_client.chat.completions.create(
                    model=current_model,
                    messages=messages_for_api_call,
                    max_tokens=max_tokens_per_turn,
                    temperature=TEMPERATURE,
                )
                ai_response_content = response.choices[0].message.content.strip()

                # Add the instruction *that led to this response* and the response itself to history
                conversation_history.append({"role": "user", "content": current_user_instruction_text})
                conversation_history.append({"role": "assistant", "content": ai_response_content})

                debate_transcript += f"**{current_model_name}:**\n{ai_response_content}\n\n---\n\n"
                time.sleep(1.5)

            except Exception as e:
                error_detail = str(e)
                # ... (keep specific error checks: Auth, RateLimit, NotFound, ContextLength) ...
                if "AuthenticationError" in error_detail: error_message_display = f"API Auth Error ({current_model_name}). Check Key."
                elif "RateLimitError" in error_detail: error_message_display = f"Rate Limit Error ({current_model_name}). Wait & retry."
                elif "NotFoundError" in error_detail and "model" in error_detail: error_message_display = f"Model Not Found ({current_model}). Check Name/Access."
                elif ("BadRequestError" in error_detail and "context_length" in error_detail) or \
                     ("invalid_request_error" in error_detail and "maximum context length" in error_detail.lower()):
                    error_message_display = f"Context Length Exceeded ({current_model_name}). Reduce turns/max_tokens or use a model with larger context."
                else: error_message_display = f"API Error ({current_model_name}): Check console."

                error_log_message = f"\n\n**API Error during {current_model_name}'s turn:** {e}\nDebate halted."
                print(error_log_message)
                debate_transcript += f"**SYSTEM:**\n*{error_message_display}*\n\n---\n\n"
                yield debate_transcript, f"Error: {current_model_name}"
                return

        final_message = "Debate Complete!"
        print(final_message)
        yield debate_transcript, final_message

    except Exception as e:
        error_message = f"\n\n**An unexpected error occurred:** {e}\nDebate halted."
        print(error_message)
        debate_transcript += f"**SYSTEM:**\n*An unexpected error occurred: {e}*\n\n---\n\n"
        yield debate_transcript, "An unexpected error occurred."

In [ ]:
# @title 3. Create and Launch Gradio Interface

# Clear any previous Gradio launches
gr.close_all()

# Define the Gradio Interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🤖 AI Debate Arena 🤺 (v5: Enhanced Interaction)
        Set debate topic, API keys, models, max tokens, and custom system prompts (personas) for each AI.
        The prompts now encourage deeper engagement with the debate history for a more immersive experience.
        **Note:** Context limits can still be reached in long debates. OpenAI usage incurs costs.
        """
    )

    with gr.Row():
        # --- Left Column: Inputs & Controls (UI layout unchanged from v4) ---
        with gr.Column(scale=1):
            topic_input = gr.Textbox(
                label="Debate Topic", placeholder="e.g., Should AI have rights?", lines=2
            )

            gr.Markdown("### Credentials")
            openai_key_input = gr.Textbox(label="OpenAI API Key", type="password", placeholder="sk-...")
            openrouter_key_input = gr.Textbox(label="OpenRouter API Key", type="password", placeholder="sk-or-...")

            gr.Markdown("### Model Selection")
            openai_model_input = gr.Textbox(label="OpenAI Model Name", value=DEFAULT_OPENAI_MODEL)
            openrouter_model_input = gr.Textbox(label="OpenRouter Model Name", value=DEFAULT_OPENROUTER_MODEL)

            gr.Markdown("### Persona / System Prompts")
            openai_system_prompt_input = gr.Textbox(
                label="OpenAI System Prompt",
                placeholder="Define OpenAI's persona/role",
                value=DEFAULT_OPENAI_SYS_PROMPT, # Uses new default
                lines=4 # Slightly more lines for longer default prompt
            )
            openrouter_system_prompt_input = gr.Textbox(
                label="OpenRouter System Prompt",
                placeholder="Define OpenRouter's persona/role",
                value=DEFAULT_OPENROUTER_SYS_PROMPT, # Uses new default
                lines=4 # Slightly more lines
            )

            gr.Markdown("### Debate Settings")
            max_tokens_slider = gr.Slider(
                label="Max Tokens per Turn", minimum=50, maximum=1500, # Increased max slightly
                step=10, value=DEFAULT_MAX_TOKENS
            )

            status_output = gr.Textbox(label="Status", placeholder="Waiting to start...", interactive=False)
            start_button = gr.Button("🚀 Start Debate", variant="primary")

        # --- Right Column: Output (Unchanged) ---
        with gr.Column(scale=2):
            debate_output = gr.Markdown(label="Debate Transcript", value="*Debate transcript will appear here...*")

    # --- Button Click Actions (Input list unchanged from v4) ---
    start_button.click(
        fn=run_debate,
        inputs=[
            topic_input,
            openai_key_input, openrouter_key_input,
            openai_model_input, openrouter_model_input,
            openai_system_prompt_input, openrouter_system_prompt_input,
            max_tokens_slider
        ],
        outputs=[debate_output, status_output]
    )

    # Examples (Unchanged)
    gr.Examples(
        examples=[
            ["Is artificial general intelligence (AGI) achievable within the next 20 years?"],
            ["Should universal basic income (UBI) be implemented globally?"],
            ["The impact of social media on mental health: net positive or negative?"]
        ],
        inputs=[topic_input], label="Example Debate Topics"
    )

# --- Launch App ---
demo.launch(share=True, debug=False)

print("\n✅ Gradio app launched!")
print("👉 Click the 'Running on public URL' link above.")
print("\n✨ New Features: Enhanced prompts aim for deeper interaction and referencing previous points.")
print("🔧 Customize personas, models, and max tokens.")
print("\n⚠️ Context window limits are still a possibility in long debates. Monitor token usage if needed.")